In [1]:
import sklearn
import numpy as np

In [2]:
def loadSamples(sample_path):
    '''
    Loads the CSV sample data
    '''
    import csv
    samples = []
    with open(sample_path + '/driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader:
            samples.append(line)
    return samples

In [3]:
def extractSamples(samples, data_path):
    '''
    Extracts needed training data and corresponding measurements
    '''
    image_paths = []
    measurements = []
    for line in samples:
        image_paths.append(data_path + '/' +line[0])    # center_image column
        measurements.append(line[3])                    # steering column
    return image_paths, measurements

In [4]:
import cv2
def generator(samples, batch_size=32):
    '''
    Generate shuffled batch samples on the fly
    '''
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        samples = sklearn.utils.shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            images = []
            steerings = []
            for image_path, measurement in batch_samples:
                image = cv2.imread(image_path)
                images.append(image)
                steerings.append(float(measurement))

                # Flip the image horizontally
                images.append(cv2.flip(image, 1))
                steerings.append(-1.0*float(measurement))
                
            X = np.array(images)
            y = np.array(steerings)
            yield sklearn.utils.shuffle(X, y)

In [5]:
data_path = '../data'
csvData = loadSamples(data_path)
image_paths, measurements = extractSamples(csvData, data_path)
samples = list(zip(image_paths, measurements))

In [6]:
import matplotlib.pyplot as plt
image_paths, measurements = extractSamples(csvData, data_path)
image = cv2.imread(image_paths[0])
top_crop = 60
bottom_crop = 25
left_crop = 0
right_crop = 0
image = cv2.rectangle(image, (left_crop,160-bottom_crop), (320 - right_crop, top_crop), color = (255, 0, 0) , thickness = 2)
print(image.shape)
cv2.imwrite('./demo_images/cropped.png', image)
cv2.imshow('image', image)
cv2.waitKey(0)
cv2.destroyAllWindows()

(160, 320, 3)


In [7]:
from sklearn.model_selection import train_test_split
train_samples, validation_samples = sklearn.model_selection.train_test_split(samples, test_size=0.2)
print('Train samples: {}'.format(len(train_samples)))
print('Validation samples: {}'.format(len(validation_samples)))

Train samples: 6428
Validation samples: 1608


In [8]:
# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

In [9]:
def nvidiaCNN(input_shape):
    '''
    Define the Nvidia End-to-End CNN architecture
    '''
    from keras.models import Sequential, Model
    from keras.layers import Lambda, Cropping2D, Convolution2D, Flatten, Dense
    model = Sequential()
    # First normalize the input image with Lambda layer
    model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=input_shape))
    # Crop the usefule area
    model.add(Cropping2D(cropping=((60,25), (0,0))))
    # 1st Convolution layer
    model.add(Convolution2D(24,5,5, subsample=(2,2), activation='relu'))
    # 2nd Convolution layer
    model.add(Convolution2D(36,5,5, subsample=(2,2), activation='relu'))  
    # 3rd Convolution layer
    model.add(Convolution2D(48,5,5, subsample=(2,2), activation='relu'))
    # 4th Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu'))
    # 5th Convolution layer
    model.add(Convolution2D(64,3,3, activation='relu'))
    # Flatten layer
    model.add(Flatten())
    # 1st Fully connected layer
    model.add(Dense(100))
    # 2nd Fully connected layer
    model.add(Dense(50))
    # 3rd Fully connected layer
    model.add(Dense(10))
    # Output layer
    model.add(Dense(1))
    print("Model Summary : \n")
    model.summary()
    return model


In [10]:
raw_image_shape = (160,320,3)
model = nvidiaCNN(raw_image_shape)

Using TensorFlow backend.
Model Summary : 

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 75, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 158, 24)       1824      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 77, 36)        21636     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 6, 37, 48)         43248     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 35, 64)         27712     
_________________________________________________________________
conv2d_5 (

In [16]:
model.compile(loss='mse', optimizer='adam')

In [18]:
history_object = model.fit_generator(train_generator, samples_per_epoch= \
                 len(train_samples), validation_data=validation_generator, \
                 nb_val_samples=len(validation_samples), nb_epoch=1, verbose=1)
model.save('model.h5')

Epoch 1/1
108/108 [==============================] - 21s 194ms/step - loss: 5.4155e-04 - val_loss: 0.0851
